# Recommender Solution Stack
The following notebook can be used to build the infrastructure necessary for serving recommmendations.
Run each step and fully review the results before proceeding.   
The entire build will take approximately 30 min

In [ ]:
import time
import json
from pprint import pprint
import boto3

In [ ]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## Gather Dataset and Solution Arn's

List existing datasets.  Datasets should have been created by cloudforamtion.

In [ ]:
personalize.list_dataset_groups()

**Attention:** Set Dataset Group ARN and solution prefix below (should be the same as stack name for consitency)

In [ ]:
# edit this
dataset_group_arn = 'arn:aws:personalize:eu-west-1:99999999:dataset-group/qa-dataset-group'
solution_stack_prefiix = 'qa'

Below are the list of recipes we will instantiate solutions thru campaigns:

## Create Solution Stack

List recipes available.

In [ ]:
personalize.list_recipes()

**Attention:** Below we assign the recipes we want to instantiate.  Edit to onboard additional recipes.

In [ ]:
# edit to add additional recipes
recipe_arns = {
               "re-rank": "arn:aws:personalize:::recipe/aws-personalized-ranking",
               "user-personalization": "arn:aws:personalize:::recipe/aws-user-personalization",
               "similar-items": "arn:aws:personalize:::recipe/aws-similar-items",
               "trending-items": 'arn:aws:personalize:::recipe/aws-trending-now'
               }

### Solutions
Now we create the solutions the actual recommemndation engines.  Note that this step will fail if solutions already exist as you can only have one solution per recipe within a dataset group.  Good order of protection.

In [ ]:
solution_arns = {}

for solution_name, arn in recipe_arns.items():
    try:
        print(f'Creating solution {solution_name}')
        response = personalize.create_solution(
        name = f'{solution_stack_prefiix}-{solution_name}',
        datasetGroupArn = dataset_group_arn,
        recipeArn = arn)

        solution_arns[solution_name] =  response['solutionArn']
        print(json.dumps(response, indent=2))
    except Exception as e:
        print(f'Solution {solution_name} failed')
        print(e)

### Event Ingestion
Event trackers are used to ingest real time interaction events

In [ ]:
tracker_response = personalize.create_event_tracker(
    name=f'{solution_stack_prefiix}-tracker',
    datasetGroupArn=dataset_group_arn
)

pprint(tracker_response)

### Solution Versions
Solution = trained models. 
Note the following steps take time so we will poll for results

In [ ]:
version_arns = {}

for solution_name, arn in solution_arns.items():
    try:
        print(f'Creating solution version {solution_name}')
        response = personalize.create_solution_version(solutionArn = arn)
        version_arns[solution_name] = response['solutionVersionArn']
        print(json.dumps(response, indent=2))
    except Exception as e:
        print(f'Solution version {solution_name} failed')
        print(e)

In [ ]:
version_arn_list = list(version_arns.values())

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for solution_version_arn in version_arn_list:
        version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = version_response["solutionVersion"]["status"]
        print(f"Status for {solution_version_arn}: {status}")
        
        if status == "ACTIVE":
            print(f"Build succeeded for {solution_version_arn}")
            version_arn_list.remove(solution_version_arn)
        elif status == "CREATE FAILED":
            print(f"Build failed for {solution_version_arn}")
            version_arn_list.remove(solution_version_arn)
    
    if len(version_arn_list) <= 0:
        break
    else:
        print("At least one solution build is still in progress")
        
    time.sleep(60)

### Campaigns
Campaign = endpoint.  Like solutions this will take time.
NOTE: this sets the minimum TPS at 1, should be changed for production workloads to avoid throttling on cold start.

In [ ]:
minimum_tps = 1 #change this value for production deploy
campaign_arns = {}

for solution_name, arn in version_arns.items():
    try:
        print(f'Creating campaign {solution_name}')
        response = personalize.create_campaign(
            name = f'{solution_stack_prefiix}-{solution_name}',
            solutionVersionArn = arn,
            minProvisionedTPS = minimum_tps
            )
        campaign_arns[solution_name] = response['campaignArn']
        print(json.dumps(response, indent=2))
    except Exception as e:
        print(f'Campaign {solution_name} failed')
        print(e)

In [ ]:
campaign_arn_list = list(campaign_arns.values())

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for campaign_arn in campaign_arn_list:
        version_response = personalize.describe_campaign(
            campaignArn = campaign_arn
        )
        status = version_response["campaign"]["status"]
        print(f"Status for {campaign_arn}: {status}")
        
        if status == "ACTIVE":
            print(f"Build succeeded for {campaign_arn}")
            campaign_arn_list.remove(campaign_arn)
        elif status == "CREATE FAILED":
            print(f"Build failed for {campaign_arn}")
            campaign_arn_list.remove(campaign_arn)
    
    if len(campaign_arn_list) <= 0:
        break
    else:
        print("At least one campaign build is still in progress")
        
    time.sleep(60)

# Teardown Commands
**DANGER!!** This will remove all campaigns and solutions.  Commented for your safety!

In [ ]:
# # set the prefix for the stack you want to delete
# solution_stack_prefiix = 'qa'

In [ ]:
# campaign_response = personalize.list_campaigns()
# pprint(campaign_response)

Iterate and delete campaigns.  This can take a few minutes.

In [ ]:
# # UNCOMMENT BELOW TO TEARDOWN

# for campaign in campaign_response['campaigns']:
#     if campaign['status'] != 'ACTIVE':
#         continue
#     campaigns_arn_delete = campaign['campaignArn']
#     if solution_stack_prefiix in campaigns_arn_delete:
#         print(f"Deleting {campaigns_arn_delete}")
#         delete_response = personalize.delete_campaign(campaignArn=campaigns_arn_delete)
#         print(delete_response)

In [ ]:
# solution_response = personalize.list_solutions()
# pprint(solution_response)

Iterate and delete solutions, this will also delete solution versions.

In [ ]:
# # UNCOMMENT BELOW TO TEARDOWN

# for solution in solution_response['solutions']:
#     if solution['status'] != 'ACTIVE':
#         continue
#     solution_arn_delete = solution['solutionArn']
#     if solution_stack_prefiix in solution_arn_delete:
#         print(f"Deleting {solution_arn_delete}")
#         delete_response = personalize.delete_solution(solutionArn=solution_arn_delete)
#         print(delete_response)
